# Intensity normalization and bit reduction

First, necessary libraries are imported and helper-functions are defined.

In [1]:
import qim3d
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import gc

def interactive_cutoff(volume: np.ndarray, cmap='gray', target_sample_size=1_000_000):
    """
    Display an interactive viewer for a 3D volume with slice navigation and intensity-cutoffs.
    Returns: (ui, range_slider) where range_slider.value has 'vmin' and 'vmax' that the user can read later.
    """
    # Subsample the volume
    stride = max(1, volume.size // target_sample_size)
    sample = volume.ravel()[::stride]

    def auto_window(method):
        if method == 'Percentile (1/99)':
            return np.percentile(sample, 1), np.percentile(sample, 99)
        elif method == 'Mean±2·Std':
            m, s = np.mean(sample), np.std(sample)
            return m - 2*s, m + 2*s
        else:
            return range_slider.value

    init_vmin, init_vmax = np.percentile(sample, 0), np.percentile(sample, 100)
    sample_min = float(sample.min())
    sample_max = float(sample.max())

    # UI elements
    slice_slider = widgets.IntSlider(
        value=volume.shape[0] // 2,
        min=0,
        max=volume.shape[0] - 1,
        description='Slice',
        continuous_update=True,
        layout=widgets.Layout(width='30%')
    )
    range_slider = widgets.FloatRangeSlider(
        value=[init_vmin, init_vmax],
        min=sample_min,
        max=sample_max,
        step=1,
        description='Intensities',
        continuous_update=True,
        layout=widgets.Layout(width='50%')
    )
    auto_dropdown = widgets.Dropdown(
        options=['Manual', 'Percentile (1/99)', 'Mean±2·Std'],
        value='Manual',
        description='Cut-off type',
    )
    log_checkbox = widgets.Checkbox(
        value=True,
        description='Log-scale on histogram',
        indent=False,
        layout=widgets.Layout(width='200px')
    )
    hist_output = widgets.Output()
    img_output = widgets.Output()

    def draw_histogram(_=None):
        with hist_output:
            hist_output.clear_output(wait=True)
            vmin, vmax = range_slider.value
            fig, ax = plt.subplots(figsize=(6, 4))
            log_state = log_checkbox.value
            ax.hist(sample, bins=64, color='orange', log=log_state)
            ax.axvline(vmin, color='red', linestyle='--', label='vmin')
            ax.axvline(vmax, color='blue', linestyle='--', label='vmax')
            ax.set_title('Approximate histogram (log)' if log_state else 'Approximate histogram')
            ax.set_xlabel('Intensity')
            ax.set_ylabel('Count (log)' if log_state else 'Count')
            plt.show()

    def update_img(_=None):
        idx = slice_slider.value
        vmin, vmax = range_slider.value
        slice_img = volume[idx]
        with img_output:
            img_output.clear_output(wait=True)
            fig, ax = plt.subplots(figsize=(6, 4))
            im = ax.imshow(
                np.clip(slice_img, vmin, vmax),
                cmap=cmap,
                vmin=vmin,
                vmax=vmax,
                interpolation='nearest',
            )
            ax.set_title(f'Slice {idx}')
            ax.axis('off')
            plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
            plt.show()

    def on_auto_dropdown_change(change):
        if change['name'] == 'value':
            new_vmin, new_vmax = auto_window(change['new'])
            # Clamp to slider range
            rng_min, rng_max = range_slider.min, range_slider.max
            new_vmin = max(rng_min, new_vmin)
            new_vmax = min(rng_max, new_vmax)
            range_slider.value = (float(new_vmin), float(new_vmax))

    def on_range_change(change):
        if change['name'] == 'value':
            update_img()       # update image for new window
            draw_histogram()   # update histogram and lines

    # Attach event listeners
    slice_slider.observe(update_img, names='value')           # image only
    range_slider.observe(on_range_change, names='value')      # state + image + hist
    auto_dropdown.observe(on_auto_dropdown_change, names='value')
    log_checkbox.observe(draw_histogram, names='value')       # histogram only

    # Initial draw
    draw_histogram()
    update_img()

    ui = widgets.VBox([
        widgets.HBox([auto_dropdown, slice_slider, range_slider]),
        log_checkbox,
        widgets.HBox([hist_output, img_output])
    ])

    return ui, range_slider

def dtype_selector(volume: np.ndarray):
    """
    Show an interactive dtype selector for a numpy array, displaying resulting sizes.
    Returns the dropdown so you can access dtype_dropdown.value for conversion.
    """
    import numpy as np
    import ipywidgets as widgets
    from IPython.display import display, HTML

    def format_size(nbytes):
        for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
            if nbytes < 10**3:
                return f"{nbytes:.2f} {unit}"
            nbytes /= 10**3
        return f"{nbytes:.2f} PB"

    def dtype_name_to_nbytes(dtype_name):
        return np.dtype(dtype_name).itemsize

    dtype_names = ['uint8', 'int8', 'uint16', 'int16', 'float16', 'float32', 'float64']
    current_size = volume.nbytes
    shape = volume.shape
    n_elements = volume.size

    options = []
    for dtype in dtype_names:
        nbytes = n_elements * dtype_name_to_nbytes(dtype)
        options.append((f"{dtype} ({format_size(nbytes)})", dtype))

    current_label = widgets.HTML(
        value=f"<b>Current dtype:</b> {volume.dtype.name} &nbsp;|&nbsp; <b>Shape:</b> {shape} &nbsp;|&nbsp; <b>Current size:</b> {format_size(current_size)}"
    )
    dtype_dropdown = widgets.Dropdown(
        options=options,
        value=volume.dtype.name,
        description='Convert to:',
        style={'description_width': 'initial'}
    )
    out = widgets.Output()

    def on_change(change):
        with out:
            out.clear_output()
            result_nbytes = n_elements * dtype_name_to_nbytes(selected_dtype)
            display(HTML(f"If converted to <b>{selected_dtype}</b>: {format_size(result_nbytes)}"))

    dtype_dropdown.observe(on_change, names='value')

    display(current_label, dtype_dropdown, out)

    # Show initial value correctly:
    with out:
        selected_dtype = dtype_dropdown.value
        result_nbytes = n_elements * dtype_name_to_nbytes(selected_dtype)
        display(HTML(f"If converted to <b>{selected_dtype}</b>: {format_size(result_nbytes)}"))

    # Return a UI handle (matching what's displayed) + live state
    ui = widgets.VBox([current_label, dtype_dropdown, out])
    return ui, dtype_dropdown

def normalize_and_convert_volume(vol_converted: np.ndarray, chosen_type: str) -> np.ndarray:
    """
    Normalize `vol_converted` and cast to `chosen_type`.
    """
    
    dtype = np.dtype(chosen_type)

    # Work in float for the normalization
    v = np.asarray(vol_converted, dtype=np.float32)

    vmin = np.nanmin(v)
    vmax = np.nanmax(v)
    span = float(vmax - vmin)


    # Handle constant volume case
    if not np.isfinite(span) or span == 0.0:
        # If float
        if dtype.kind in ('f',):
            out = np.zeros_like(v, dtype=dtype)
            print('Volume intensities are now in range [0, 1]')
            return out
        # If integer
        elif dtype.kind in ('i', 'u'):
            info = np.iinfo(dtype)
            out = np.zeros_like(v, dtype=dtype)  # choose 0 in all integer ranges
            print(f'Volume intensities are now in range [{info.min}, {info.max}]')
            return out
        else:
            raise ValueError("Invalid chosen_type")

    # Normalize in-place to [0,1] (would be: v = (v - vmin) / span)
    np.subtract(v, vmin, out=v)
    np.divide(v, span, out=v)

    # If float
    if dtype.kind in ('f',):
        out = v.astype(dtype, copy=False)
        print('Volume intensities are now in range [0, 1]')
        return out

    # If integer
    elif dtype.kind in ('i', 'u'):
        # Map [0,1] to [info.min, info.max] in-place
        info = np.iinfo(dtype)
        rng = float(info.max - info.min)

        # v = v * rng + info.min
        np.multiply(v, rng, out=v)
        np.add(v, float(info.min), out=v)

        # Round to nearest int and clip
        np.rint(v, out=v)

        # Replace NaNs with 0.0 before casting (cannot cast NaN to int)
        np.nan_to_num(v, copy=False, nan=0.0, posinf=float(info.max), neginf=float(info.min))
        np.clip(v, float(info.min), float(info.max), out=v)

        # Final cast
        out = v.astype(dtype)
        print(f'Volume intensities are now in range [{info.min}, {info.max}]')
        return out

    else:
        raise ValueError("Invalid chosen_type")

The volumetric data is now loaded in.

In [2]:
downloader = qim3d.io.Downloader()
vol = downloader.Snail.Escargot(load_file=True)

# The below is not normally needed if loading in your volume using qim3d.io.load() !

# Convert to a normal ndarray in native byte order (safe for plotting & processing)
if vol.dtype.byteorder not in ('=', '|'):                   # not native-endian
    vol = vol.byteswap().view(vol.dtype.newbyteorder())     # makes little-endian
vol = np.ascontiguousarray(vol) 

orig_diskspace = vol.size * vol.itemsize / 10**9
print(20 * "-")
print(f"The volume currently takes up {orig_diskspace} GB of disk space")

https://archive.compute.dtu.dk/download/public/projects/viscomp_data_repository/Snail/Escargot.tif
2.61GB [02:10, 21.5MB/s]                                                        

Loading Escargot.tif
Using virtual stack


--------------------
The volume currently takes up 2.8 GB of disk space


## Voxel intensity cut-off

We wish to gain insights into the voxel intensity distribution of the volume. To do this, an approximate histogram is created. The range of voxel intensities can be changed interactively to remove unnecessary values so that only intensities that are relevant for the analysis retain.

In [3]:
ui, slider = interactive_cutoff(vol)
display(ui)

Now the cut-off values can be applied to the volume.

In [4]:
vmin, vmax = slider.value
vol_clipped = np.clip(vol, vmin, vmax)

## Selecting appropriate bit depth

Volumetric data from micro-CT-scanners and synchrotrons are often saved with unnecessarily high number precision, making it take up unnecessary amounts of disk space.

Below you can change to other bit depths and see the new amount of disk space it will take up.

In [5]:
ui, dropdown = dtype_selector(vol)

HTML(value='<b>Current dtype:</b> float32 &nbsp;|&nbsp; <b>Shape:</b> (700, 1000, 1000) &nbsp;|&nbsp; <b>Curre…

Dropdown(description='Convert to:', index=5, options=(('uint8 (700.00 MB)', 'uint8'), ('int8 (700.00 MB)', 'in…

Output()

Voxel intensities from micro-CT-scanners and synchrotrons often come in arbitrary ranges. The range will now be fixed to the range corresponding to the chosen bit depth.

In [6]:
chosen_type = dropdown.value

# Convert volume
vol_normalized = normalize_and_convert_volume(vol_clipped, chosen_type)
new_diskspace = vol_normalized.size * vol_normalized.itemsize / 10**9

# Free memory
del vol_clipped
gc.collect()

print(f"Volume now takes up {new_diskspace} GB of disk space - {(new_diskspace/orig_diskspace) * 100}% of the original")

Volume intensities are now in range [-128, 127]
Volume now takes up 0.7 GB of disk space - 25.0% of the original


## Visual checks

Below the new volume and original volume can be inspected next to eachother.

In [7]:
ui_new  = qim3d.viz.slicer(vol_normalized, color_map='gray')
ui_orig = qim3d.viz.slicer(vol,           color_map='gray')

label_new  = widgets.Label("New volume",     layout=widgets.Layout(align_self='center'))
label_orig = widgets.Label("Original volume", layout=widgets.Layout(align_self='center'))

col1 = widgets.VBox([label_new, ui_new])
col2 = widgets.VBox([label_orig, ui_orig])

display(widgets.HBox([col1, col2]))

## Exporting volume

The new volume can now be exported. The file format and spot for saving is chosen by the path which you can change below.

In [8]:
path = './vol_normalized.tif'

qim3d.io.save(path, vol_normalized)